# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
import gmaps.datasets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
weather_data = pd.read_csv('../WeatherPy/output_data/cities.csv')
weather_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.8000,-139.0333,77.49,81,5,15.77,PF,1610408207
1,1,ribeira grande,38.5167,-28.7000,62.53,89,100,20.11,PT,1610408207
2,2,rikitea,-23.1203,-134.9692,78.40,77,95,18.39,PF,1610408208
3,3,vila velha,-20.3297,-40.2925,79.00,83,75,10.36,BR,1610408208
4,4,sinnamary,5.3833,-52.9500,77.68,85,43,12.62,GF,1610408208
...,...,...,...,...,...,...,...,...,...,...
537,537,cayenne,4.9333,-52.3333,77.00,100,75,5.75,GF,1610408273
538,538,raton,36.9034,-104.4391,37.40,41,1,8.05,US,1610408125
539,539,sidrolandia,-20.9319,-54.9614,82.40,61,40,6.91,BR,1610408273
540,540,gananoque,44.3334,-76.1661,32.00,1,100,11.18,CA,1610408273


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
gmaps.configure(g_key)

In [17]:
#Store latitude and longitude in locations
locations = weather_data[['Lat', 'Lng']]

humidity = weather_data["Humidity"]

In [18]:
#Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_humidity = np.max(humidity)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3, opacity=0.5)
fig.add_layer(heat_layer)
fig

plt.savefig("output_data/heatmap.png")

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#Narrow the dataframe
narrowed_city_df = weather_data.loc[(weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80) & (weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0)].dropna()
narrowed_city_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
#Create a hotel dataframe
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
         "keyword" : "hotel",
         "radius" : 5000,
         "key" : g_key}

In [ ]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig